# *Desulfonema ishimotonii* CASP-σ Motif Search

## Approach

3 tiers of searching: DiCASP locus, promoters, and the entire genome for other possibilities

- DiCASP locus search: motif constructed with corrected background frequencies, direct search in FIMO

- Promoter search: pre-filter promoters for all CDSs (-100 to +1), and then scan in FIMO

- Genome search: motif constructed with corrected background frequencies, direct search in FIMO

In [1]:
# Import Libraries
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

## DiCasp Locus Search

Background frequencies in the loci were: 

|     |Freq    |%     |
|-----|--------|------|
|A:   |5,362   |26.4% |
|C:   |4,750   |23.3% |
|G:   |5,388   |26.5% |
|T:   |4,845   |23.8% |
|GC:  |10,138  |49.8% |
|All: |20,345  |100.0%|
 

FIMO with a p-value threshold of 1e-4 found 3 hits, all significant with low q-values.

In [2]:
# Read in hits
LociHits = pd.read_csv("fimo_DiCASPLoci.tsv", sep='\t', header=0)[:-3] # Last 3 rows are run info
del LociHits["motif_id"], LociHits["motif_alt_id"], LociHits["sequence_name"]
LociHits.to_csv("Loci_Motif_Search_Hits.csv")
LociHits

,start,stop,strand,score,p-value,q-value,matched_sequence
0,6650.0,6673.0,+,19.3776,6.830000e-08,0.00278,TCACATTTCCGAAAAAAGCGCGAC
1,1377.0,1400.0,+,19.0714,1.370000e-07,0.00279,TCACATTTTCCGAAAACGTGCGAC
2,7683.0,7706.0,+,17.5306,8.150000e-07,0.01100,TCACATTCTGATTTTTATTACGAC


## Promoter Search

Pre-filter promoters for all CDSs (-100 to +1), and then scan in FIMO

In [3]:
# Extract Promoter Sequences

DiGenome_record = SeqIO.read("Desulfonema_ishimotonii_strain_Tokyo_01_NODE_1_-_NZ_BEXT01000001_DiGenome.gb", "genbank")
promoter_list = []
promoter_records = []

# Iterate through CDSs and store positions/names
# All CDSs are stored as a hittuple of (DENIS_name, product, (start, stop, strand))
for feature in DiGenome_record.features:
    if feature.type == "CDS":
        feature_list = [feature.qualifiers.get("locus_tag")[0], feature.qualifiers.get("product")[0]]
        if feature.strand == -1:
            feature_list.append((feature.location.end-1, feature.location.end+100, "-"))
        elif feature.strand == 1:
            feature_list.append((max(feature.location.start-100, 0), feature.location.start+1, "+"))
        promoter_list.append(tuple(feature_list))
        
# Export into .fasta format
for i, hittuple in enumerate(promoter_list):
    sequence = DiGenome_record.seq[hittuple[2][0]:hittuple[2][1]]
    promoter_records.append(
        SeqRecord(sequence, 
                  id="%s %s" % (hittuple[0], hittuple[1]), 
                  description="%d-%d %s" % (hittuple[2][0], hittuple[2][1], hittuple[2][2]))
    )
    
SeqIO.write(promoter_records, open("DiPromoters.fasta", "w"), "fasta")

5087

Background frequencies in this promoter list: 

|     |Freq    |%     |
|-----|--------|------|
|A:   |133,967 |26.1% |
|C:   |120,499 |23.5% |
|G:   |121,121 |23.6% |
|T:   |138,155 |26.9% |
|GC:  |241,620 |47.0% |
|All: |513,742 |100.0%|

FIMO with a p-value threshold of 1e-4 found 44 hits, only 1 additional significant with a q-value under 0.6.

In [4]:
# Read in hits and export table
PromoterHits = pd.read_csv("fimo_DiPromoters.tsv", sep='\t', header=0)[:-3] # Last 3 rows are run info
del PromoterHits["motif_id"], PromoterHits["motif_alt_id"]
PromoterHits.to_csv("Promoter_Motif_Search_Hits.csv")
PromoterHits

,sequence_name,start,stop,strand,score,p-value,q-value,matched_sequence
0,DENIS_1075,35.0,58.0,+,19.02040,1.330000e-07,0.0996,TCACATTTTCCGAAAACGTGCGAC
1,DENIS_1077,5.0,28.0,+,18.69390,2.510000e-07,0.0996,TCACATTCTGATTTTTATTACGAC
2,DENIS_0717,34.0,57.0,+,16.31630,2.090000e-06,0.5520,CAACATTCCACCACATCAGGCGAC
3,DENIS_3089,11.0,34.0,+,15.62240,4.010000e-06,0.7960,TCACAATGTATGAAATCACACCAC
4,DENIS_4103,21.0,44.0,-,13.86730,1.080000e-05,1.0000,TCACATCCCAGCGTCCCGGCCGAT
5,DENIS_3478,25.0,48.0,+,13.74490,1.150000e-05,1.0000,TCACATCACAATGGCAGCGGCCAC
6,DENIS_0717,24.0,47.0,+,13.69390,1.180000e-05,1.0000,TAACAATTTTCAACATTCCACCAC
7,DENIS_1114,32.0,55.0,-,13.40820,1.380000e-05,1.0000,CAACATTTCGTCAAGACATGCGAT
8,DENIS_4298,47.0,70.0,-,13.39800,1.390000e-05,1.0000,TAACATTGGGATAACAGCTCTGAC
9,DENIS_1627,54.0,77.0,-,13.24490,1.510000e-05,1.0000,TCCCATATATTGTTCTTTGACGAC


## Full Genome Search

Background frequncies in the genome were

|     |Freq      |%     |
|-----|----------|------|
|A:   |1,524,593 |23.1% |
|C:   |1,765,541 |26.7% |
|G:   |1,773,258 |26.8% |
|T:   |1,773,258 |23.4% |
|GC:  |3,538,799 |53.5% |
|All: |6,610,564 |100.0%|
 
FIMO with a p-value threshold of 1e-4 found 1036 hits, unclear how to interpret significance.
Especially as it is unclear what binding inside CDSs would do.

In [5]:
# Read in hits and export table
GenomeHits = pd.read_csv("fimo_DiGenome.tsv", sep='\t', header=0)[:-3] # Last 3 rows are run info
del GenomeHits["motif_id"], GenomeHits["motif_alt_id"], GenomeHits["sequence_name"]
GenomeHits.to_csv("Genome_Motif_Search_Hits.csv")
GenomeHits

,start,stop,strand,score,p-value,q-value,matched_sequence
0,471166.0,471189.0,+,20.78570,3.350000e-09,0.0442,TCACATTGCCCCGGATGTGCCGAC
1,747240.0,747263.0,+,19.88780,2.670000e-08,0.1770,CCACATTATGAGCCGGGGGGCGAC
2,4704281.0,4704304.0,-,18.88780,1.910000e-07,0.6160,CCACATTGACCACGGCAAGACGAC
3,1368036.0,1368059.0,+,18.84690,2.210000e-07,0.6160,TCACATTTTCCGAAAACGTGCGAC
4,1373309.0,1373332.0,+,18.80610,2.330000e-07,0.6160,TCACATTTCCGAAAAAAGCGCGAC
...,...,...,...,...,...,...,...
1031,310408.0,310431.0,+,8.40816,9.990000e-05,1.0000,CCCCATCCGGAGGCGGACCACCAT
1032,2046308.0,2046331.0,+,8.40816,9.990000e-05,1.0000,TCACATTACAAGCTGGACGGCCAG
1033,2615128.0,2615151.0,-,8.40816,9.990000e-05,1.0000,CAACATGCAGGTCTGCAACCCGAC
1034,3392419.0,3392442.0,+,8.40816,9.990000e-05,1.0000,TCACAAAAAAAGAATCAGACAGAT
